# Monte-Carlo Off-Police

### Instalação de pacotes

In [9]:
from IPython.display import clear_output
!apt-get install ffmpeg freeglut3-dev xvfb  
!pip install gym[all]==00.25.1
!pip install gym[atari,accept-rom-license]==00.25.1
!pip install pyglet
!pip install stable-baselines3[extra]
!pip install optuna
clear_output()

In [10]:
!mkdir log_project

mkdir: cannot create directory ‘log_project’: File exists


### Imports


In [11]:
import gym
import numpy as np
import tensorboard
import matplotlib.pyplot as plt
import time

%load_ext tensorboard

import gym
import numpy as np


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Para salvar vídeo

In [12]:
# Set up fake display; otherwise rendering will fail
import os
!pip install stable-baselines3[extra]
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


A gravação é feita com o wrapper [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder).

In [13]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record the given number of steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [14]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [60]:
# ideias adaptadas de : https://www.anyscale.com/blog/an-introduction-to-reinforcement-learning-with-openai-gym-rllib-and-google
from base64 import b64encode
from IPython.display import HTML
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  html_code = f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'
  return HTML(html_code)

# Código

In [15]:
ENV_NAME = "Taxi-v3"  
 
env = gym.make(ENV_NAME)

## Off-policy

In [41]:
def choose_action(Q, state):
    return np.argmax(Q[state])

def choose_actionB(num_actions):
    return np.random.randint(0, num_actions)

# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarloOffP(env, episodes, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    
    num_actions = env.action_space.n
    
    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))
    C = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []
        
        state = env.reset()
    
        # PARTE 1: executa um episódio completo
        while done != True:   
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios 
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = choose_action(Q, state)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )
            
            sum_rewards += reward
            state = next_state
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # PARTE 2: atualiza Q (e a política, implicitamente)
        Gt = 0
        W = 1
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            C[s,a] = C[s,a] + W
            delta = W * (Gt - Q[s,a])
            Q[s,a] = Q[s,a] + (1/C[s,a])* delta
            best = choose_action(Q,s)
            if best != choose_actionB(num_actions): break
            W = W*(1/(1/num_actions))

    return sum_rewards_per_ep, Q


## On-Policy

In [34]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def pi_policy(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])


# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarloOnP(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    
    num_actions = env.action_space.n
    
    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []
        
        state = env.reset()
    
        # PARTE 1: executa um episódio completo
        while done != True:   
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios 
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = pi_policy(Q, state, num_actions, epsilon)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )
            
            sum_rewards += reward
            state = next_state
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # PARTE 2: atualiza Q (e a política, implicitamente)
        Gt = 0
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            delta = Gt - Q[s,a]
            Q[s,a] = Q[s,a] + lr * delta

    return sum_rewards_per_ep, Q

### Execução Off-Policy

### Otimiza Parâmetros

In [18]:
from numpy.random.mtrand import gamma

import optuna



ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    #bins1 = trial.suggest_int('bins1', 5, 100)
    #bins2 = trial.suggest_int('bins2', 5, 100)
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    #env_wrapper = DiscreteObservationWrapper(ENV, [bins1,bins2])
    (returns, _) = run_montecarloOffP(env, 20000, gamma, eps, render=False)
    return sum(returns[-100:])/100 

In [19]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20) 

[I 2022-10-02 20:05:44,145] Using an existing study with name 'new_MC_offpolice' instead of creating a new one.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  app.launch_new_instance()



TRIAL #1: eps=0.16324236563466543, gamma=0.579879263191128
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -289.730
Episode 300 Average Reward (last 100): -307.460
Episode 400 Average Reward (last 100): -468.560
Episode 500 Average Reward (last 100): -576.020
Episode 600 Average Reward (last 100): -827.930
Episode 700 Average Reward (last 100): -827.930
Episode 800 Average Reward (last 100): -954.470
Episode 900 Average Reward (last 100): -864.290
Episode 1000 Average Reward (last 100): -954.560
Episode 1100 Average Reward (last 100): -1080.110
Episode 1200 Average Reward (last 100): -954.560
Episode 1300 Average Reward (last 100): -810.110
Episode 1400 Average Reward (last 100): -809.930
Episode 1500 Average Reward (last 100): -737.390
Episode 1600 Average Reward (last 100): -756.380
Episode 1700 Average Reward (last 100): -559.010
Episode 1800 Average Reward (last 100): -666.650
Episode 1900 Average Reward (last 100): -522.830
Episode 2000 Aver

[I 2022-10-02 20:06:48,326] Trial 1 finished with value: -200.0 and parameters: {'gamma': 0.579879263191128, 'epsilon': 0.16324236563466543}. Best is trial 1 with value: -200.0.


Episode 20000 Average Reward (last 100): -200.000

TRIAL #2: eps=0.04986517974023241, gamma=0.35524532169751455
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -289.190
Episode 300 Average Reward (last 100): -360.290
Episode 400 Average Reward (last 100): -432.380
Episode 500 Average Reward (last 100): -630.290
Episode 600 Average Reward (last 100): -703.280
Episode 700 Average Reward (last 100): -916.850
Episode 800 Average Reward (last 100): -666.830
Episode 900 Average Reward (last 100): -882.200
Episode 1000 Average Reward (last 100): -1044.020
Episode 1100 Average Reward (last 100): -898.670
Episode 1200 Average Reward (last 100): -1043.390
Episode 1300 Average Reward (last 100): -899.930
Episode 1400 Average Reward (last 100): -864.470
Episode 1500 Average Reward (last 100): -792.020
Episode 1600 Average Reward (last 100): -666.560
Episode 1700 Average Reward (last 100): -630.290
Episode 1800 Average Reward (last 100): -486.650
Episode 1900 

[I 2022-10-02 20:07:43,611] Trial 2 finished with value: -217.64 and parameters: {'gamma': 0.35524532169751455, 'epsilon': 0.04986517974023241}. Best is trial 1 with value: -200.0.


...

Episode 20000 Average Reward (last 100): -234.740

TRIAL #19: eps=0.18091385227838208, gamma=0.823214864869089
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -235.730
Episode 300 Average Reward (last 100): -289.460
Episode 400 Average Reward (last 100): -540.200
Episode 500 Average Reward (last 100): -647.930
Episode 600 Average Reward (last 100): -594.470
Episode 700 Average Reward (last 100): -863.750
Episode 800 Average Reward (last 100): -810.740
Episode 900 Average Reward (last 100): -756.920
Episode 1000 Average Reward (last 100): -972.830
Episode 1100 Average Reward (last 100): -1043.390
Episode 1200 Average Reward (last 100): -1008.110
Episode 1300 Average Reward (last 100): -1170.200
Episode 1400 Average Reward (last 100): -828.650
Episode 1500 Average Reward (last 100): -971.840
Episode 1600 Average Reward (last 100): -918.110
Episode 1700 Average Reward (last 100): -612.650
Episode 1800 Average Reward (last 100): -648.110
Episode 1900 

[I 2022-10-02 20:23:08,847] Trial 19 finished with value: -197.9 and parameters: {'gamma': 0.823214864869089, 'epsilon': 0.18091385227838208}. Best is trial 16 with value: -168.71.


Episode 20000 Average Reward (last 100): -197.900

TRIAL #20: eps=0.13697833152636996, gamma=0.5548420226907784
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -253.370
Episode 300 Average Reward (last 100): -396.830
Episode 400 Average Reward (last 100): -433.100
Episode 500 Average Reward (last 100): -701.390
Episode 600 Average Reward (last 100): -773.660
Episode 700 Average Reward (last 100): -667.100
Episode 800 Average Reward (last 100): -828.650
Episode 900 Average Reward (last 100): -918.110
Episode 1000 Average Reward (last 100): -1008.020
Episode 1100 Average Reward (last 100): -900.650
Episode 1200 Average Reward (last 100): -1043.750
Episode 1300 Average Reward (last 100): -881.300
Episode 1400 Average Reward (last 100): -918.020
Episode 1500 Average Reward (last 100): -828.380
Episode 1600 Average Reward (last 100): -594.920
Episode 1700 Average Reward (last 100): -738.110
Episode 1800 Average Reward (last 100): -684.200
Episode 1900 

[I 2022-10-02 20:24:08,244] Trial 20 finished with value: -200.0 and parameters: {'gamma': 0.5548420226907784, 'epsilon': 0.13697833152636996}. Best is trial 16 with value: -168.71.


Episode 20000 Average Reward (last 100): -200.000


### Execução

In [43]:

env = gym.make("Taxi-v3")

In [46]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.01
    GAMMA = 0.830525147061507
    EPSILON = 0.05919712699520377

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOffP(env, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffP-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'offPolicyD')

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

Episode 100 Average Reward (last 100): -217.820
Episode 200 Average Reward (last 100): -307.010
Episode 300 Average Reward (last 100): -342.110
Episode 400 Average Reward (last 100): -572.510
Episode 500 Average Reward (last 100): -571.250
Episode 600 Average Reward (last 100): -481.610
Episode 700 Average Reward (last 100): -603.560
Episode 800 Average Reward (last 100): -462.440
Episode 900 Average Reward (last 100): -533.900
Episode 1000 Average Reward (last 100): -484.130
Episode 1100 Average Reward (last 100): -396.020
Episode 1200 Average Reward (last 100): -395.750
Episode 1300 Average Reward (last 100): -393.230
Episode 1400 Average Reward (last 100): -640.190
Episode 1500 Average Reward (last 100): -482.960
Episode 1600 Average Reward (last 100): -445.790
Episode 1700 Average Reward (last 100): -411.410
Episode 1800 Average Reward (last 100): -252.830
Episode 1900 Average Reward (last 100): -235.910
Episode 2000 Average Reward (last 100): -341.930
Episode 2100 Average Reward (

### Execução On-Policy

#### Otimiza Parâmetros

In [21]:
from numpy.random.mtrand import gamma

import optuna



ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('learning_rate', 0.001, 1.0)
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
   
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    
    (returns, _) = run_montecarloOnP(ENV, 20000, lr, gamma, eps, render=False)
    return sum(returns[-100:])/100 

In [22]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_onpolice', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20) 

[I 2022-10-02 20:28:52,242] A new study created in RDB with name: new_MC_onpolice
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



TRIAL #0: eps=0.1609010081094748, gamma=0.4221866878016252
Episode 100 Average Reward (last 100): -438.800
Episode 200 Average Reward (last 100): -552.890
Episode 300 Average Reward (last 100): -332.100
Episode 400 Average Reward (last 100): -326.810
Episode 500 Average Reward (last 100): -322.940
Episode 600 Average Reward (last 100): -297.880
Episode 700 Average Reward (last 100): -304.430
Episode 800 Average Reward (last 100): -284.980
Episode 900 Average Reward (last 100): -305.570
Episode 1000 Average Reward (last 100): -296.570
Episode 1100 Average Reward (last 100): -283.950
Episode 1200 Average Reward (last 100): -282.900
Episode 1300 Average Reward (last 100): -297.010
Episode 1400 Average Reward (last 100): -279.750
Episode 1500 Average Reward (last 100): -282.970
Episode 1600 Average Reward (last 100): -296.480
Episode 1700 Average Reward (last 100): -290.780
Episode 1800 Average Reward (last 100): -282.830
Episode 1900 Average Reward (last 100): -286.990
Episode 2000 Avera

[I 2022-10-02 20:29:53,774] Trial 0 finished with value: -280.63 and parameters: {'learning_rate': 0.9513257195608331, 'gamma': 0.4221866878016252, 'epsilon': 0.1609010081094748}. Best is trial 0 with value: -280.63.


Episode 20000 Average Reward (last 100): -280.630

TRIAL #1: eps=0.09637271526233701, gamma=0.5743797421678166
Episode 100 Average Reward (last 100): -468.290
Episode 200 Average Reward (last 100): -634.070
Episode 300 Average Reward (last 100): -541.820
Episode 400 Average Reward (last 100): -330.410
Episode 500 Average Reward (last 100): -351.020
Episode 600 Average Reward (last 100): -260.030
Episode 700 Average Reward (last 100): -260.660
Episode 800 Average Reward (last 100): -261.830
Episode 900 Average Reward (last 100): -265.070
Episode 1000 Average Reward (last 100): -261.110
Episode 1100 Average Reward (last 100): -271.820
Episode 1200 Average Reward (last 100): -255.170
Episode 1300 Average Reward (last 100): -261.200
Episode 1400 Average Reward (last 100): -262.280
Episode 1500 Average Reward (last 100): -255.440
Episode 1600 Average Reward (last 100): -260.210
Episode 1700 Average Reward (last 100): -257.510
Episode 1800 Average Reward (last 100): -264.080
Episode 1900 Ave

[I 2022-10-02 20:30:52,283] Trial 1 finished with value: -204.39 and parameters: {'learning_rate': 0.24544278326946914, 'gamma': 0.5743797421678166, 'epsilon': 0.09637271526233701}. Best is trial 1 with value: -204.39.


...

Episode 20000 Average Reward (last 100): -103.790

TRIAL #18: eps=0.15253182902235088, gamma=0.7120895256250555
Episode 100 Average Reward (last 100): -479.990
Episode 200 Average Reward (last 100): -652.520
Episode 300 Average Reward (last 100): -551.540
Episode 400 Average Reward (last 100): -420.410
Episode 500 Average Reward (last 100): -331.220
Episode 600 Average Reward (last 100): -330.470
Episode 700 Average Reward (last 100): -316.390
Episode 800 Average Reward (last 100): -297.770
Episode 900 Average Reward (last 100): -298.900
Episode 1000 Average Reward (last 100): -269.730
Episode 1100 Average Reward (last 100): -285.090
Episode 1200 Average Reward (last 100): -290.820
Episode 1300 Average Reward (last 100): -276.710
Episode 1400 Average Reward (last 100): -269.920
Episode 1500 Average Reward (last 100): -289.070
Episode 1600 Average Reward (last 100): -264.370
Episode 1700 Average Reward (last 100): -262.160
Episode 1800 Average Reward (last 100): -268.760
Episode 1900 Av

[I 2022-10-02 20:44:38,392] Trial 18 finished with value: -170.79 and parameters: {'learning_rate': 0.1288231270602922, 'gamma': 0.7120895256250555, 'epsilon': 0.15253182902235088}. Best is trial 11 with value: -1.33.



TRIAL #19: eps=0.12209385340171718, gamma=0.8641507048437724
Episode 100 Average Reward (last 100): -437.420
Episode 200 Average Reward (last 100): -567.470
Episode 300 Average Reward (last 100): -461.540
Episode 400 Average Reward (last 100): -363.800
Episode 500 Average Reward (last 100): -343.790
Episode 600 Average Reward (last 100): -289.820
Episode 700 Average Reward (last 100): -297.740
Episode 800 Average Reward (last 100): -281.630
Episode 900 Average Reward (last 100): -273.980
Episode 1000 Average Reward (last 100): -275.150
Episode 1100 Average Reward (last 100): -282.260
Episode 1200 Average Reward (last 100): -279.470
Episode 1300 Average Reward (last 100): -279.200
Episode 1400 Average Reward (last 100): -273.260
Episode 1500 Average Reward (last 100): -275.330
Episode 1600 Average Reward (last 100): -278.430
Episode 1700 Average Reward (last 100): -272.260
Episode 1800 Average Reward (last 100): -265.430
Episode 1900 Average Reward (last 100): -269.840
Episode 2000 Ave

[I 2022-10-02 20:45:33,410] Trial 19 finished with value: -239.22 and parameters: {'learning_rate': 0.5027795200258759, 'gamma': 0.8641507048437724, 'epsilon': 0.12209385340171718}. Best is trial 11 with value: -1.33.


Episode 20000 Average Reward (last 100): -239.220


#### Execução

In [47]:
ENV_NAME = "Taxi-v3"  
#ENV_NAME = "MountainCarContinuous-v0"  
#ENV_NAME = "LunarLander-v2"  
env = gym.make(ENV_NAME)

#parameters: {'learning_rate': 0.007617314337157292, 'gamma': 0.8762195844017038, 'epsilon': 0.1594319424288697}. Best is trial 16 with value: -2.2.
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.007617314337157292
    GAMMA = 0.8762195844017038
    EPSILON = 0.1594319424288697

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOnP(env, EPISODES, LR, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOnP-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'onPolicyD')

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

Episode 100 Average Reward (last 100): -490.880
Episode 200 Average Reward (last 100): -933.430
Episode 300 Average Reward (last 100): -989.570
Episode 400 Average Reward (last 100): -806.360
Episode 500 Average Reward (last 100): -558.750
Episode 600 Average Reward (last 100): -451.030
Episode 700 Average Reward (last 100): -470.900
Episode 800 Average Reward (last 100): -391.900
Episode 900 Average Reward (last 100): -437.080
Episode 1000 Average Reward (last 100): -417.480
Episode 1100 Average Reward (last 100): -308.200
Episode 1200 Average Reward (last 100): -316.640
Episode 1300 Average Reward (last 100): -321.960
Episode 1400 Average Reward (last 100): -262.400
Episode 1500 Average Reward (last 100): -311.510
Episode 1600 Average Reward (last 100): -251.250
Episode 1700 Average Reward (last 100): -226.730
Episode 1800 Average Reward (last 100): -220.300
Episode 1900 Average Reward (last 100): -220.680
Episode 2000 Average Reward (last 100): -210.440
Episode 2100 Average Reward (

## Contínuo

In [51]:

ENV_NAME = "MountainCar-v0"  

env = gym.make(ENV_NAME)

In [25]:

class GeneralDiscretizer:
    def __init__(self, env, bins_per_dimension):
        self.bins_per_dim = bins_per_dimension.copy()
        self.intervals_per_dim = []
        self.total_bins = 1
        for i, bins in enumerate(bins_per_dimension):
            self.intervals_per_dim.append(
                np.linspace(env.observation_space.low[i], env.observation_space.high[i], bins+1) )
            self.total_bins *= bins

    def to_single_bin(self, state):
        bin_vector = [(np.digitize(x=state[i], bins=intervals) - 1)
                      for i, intervals in enumerate(self.intervals_per_dim)]
        # print(bin_vector)
        return self._bin_vector_to_single_bin(bin_vector, len(bin_vector)-1)

    def _bin_vector_to_single_bin(self, vector, index):
        if index < 0:
            return 0
        return vector[index] + self.bins_per_dim[index] * self._bin_vector_to_single_bin(vector, index-1)

    def get_total_bins(self):
        return self.total_bins


class DiscreteObservationWrapper(gym.ObservationWrapper):
    '''Classe para converter espaços contínuos em espaços discretos.
    Esta classe converte ambientes de observações (estados) contínuos em ambientes de estados
    discretos. Especificamente, ele converte representações dadas na forma de array de valores float
    em um único inteiro $\geq$ não-negativo (>=0).
    
    Precisa passar para o construtor uma lista que informa em quantos "bins" vai ser discretizada 
    cada dimensão (ou seja, cada valor float) do espaço de estados original.
    '''
    
    def __init__(self, env, BINS_PER_DIMENSION):
        super().__init__(env)
        # cria um GeneralDiscretizer para converter um array de valores float em um único inteiro >= 0
        # precisa dizer em quantos "bins" vai ser discretizada cada dimensão
        self.discretizer = GeneralDiscretizer(env, BINS_PER_DIMENSION)
        self.observation_space = gym.spaces.Discrete(self.discretizer.get_total_bins())

    def observation(self, obs):
        return self.discretizer.to_single_bin(obs)


In [54]:
from numpy.random.mtrand import gamma

import optuna



# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_valuesOf(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    bins1 = trial.suggest_int('bins1', 5.0, 80.0)
    bins2 = trial.suggest_int('bins2', 10.0, 90.0)
    
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}, bins1={bins1},bins2={bins2}")

    # roda o algoritmo e recebe os retornos não-descontados
    env_wrapper = DiscreteObservationWrapper(env, [bins1,bins2])
    (returns, _) = run_montecarloOffP(env_wrapper, 20000, gamma,eps, render=False)
    return sum(returns[-100:])/100

In [55]:
def train_valuesOn(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    bins1 = trial.suggest_int('bins1', 5.0, 80.0)
    bins2 = trial.suggest_int('bins2', 10.0, 90.0)
   
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}, bins1={bins1},bins2={bins2}")

    # roda o algoritmo e recebe os retornos não-descontados
    env_wrapper = DiscreteObservationWrapper(env, [bins1,bins2])
    (returns, _) = run_montecarloOffP(env_wrapper, 20000, gamma, eps, render=False)
    return sum(returns[-100:])/100

In [52]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_cont_bins_on-Policy', 
                            load_if_exists=True)


study2 = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_cont_bins_off-policy', 
                            load_if_exists=True)


study.optimize(train_valuesOn, n_trials=30) 


[I 2022-10-03 00:09:40,314] Using an existing study with name 'new_MC_offpolice_cont_bins_on-Policy' instead of creating a new one.
[I 2022-10-03 00:09:40,371] Using an existing study with name 'new_MC_offpolice_cont_bins_off-policy' instead of creating a new one.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """



TRIAL #24: eps=0.01243910657028749, gamma=0.17709806307166054, bins1=25,bins2=34
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -200.000
Episode 300 Average Reward (last 100): -200.000
Episode 400 Average Reward (last 100): -200.000
Episode 500 Average Reward (last 100): -200.000
Episode 600 Average Reward (last 100): -200.000
Episode 700 Average Reward (last 100): -200.000
Episode 800 Average Reward (last 100): -200.000
Episode 900 Average Reward (last 100): -200.000
Episode 1000 Average Reward (last 100): -200.000
Episode 1100 Average Reward (last 100): -200.000
Episode 1200 Average Reward (last 100): -200.000
Episode 1300 Average Reward (last 100): -200.000
Episode 1400 Average Reward (last 100): -200.000
Episode 1500 Average Reward (last 100): -200.000
Episode 1600 Average Reward (last 100): -200.000
Episode 1700 Average Reward (last 100): -200.000
Episode 1800 Average Reward (last 100): -200.000
Episode 1900 Average Reward (last 100): -200.

[I 2022-10-03 00:14:30,119] Trial 24 finished with value: -200.0 and parameters: {'gamma': 0.17709806307166054, 'epsilon': 0.01243910657028749, 'bins1': 25, 'bins2': 34}. Best is trial 7 with value: -199.26.


Episode 20000 Average Reward (last 100): -200.000

TRIAL #25: eps=0.134119152413946, gamma=0.6469940384249309, bins1=65,bins2=72
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -200.000
Episode 300 Average Reward (last 100): -200.000
Episode 400 Average Reward (last 100): -200.000
Episode 500 Average Reward (last 100): -200.000
Episode 600 Average Reward (last 100): -200.000
Episode 700 Average Reward (last 100): -200.000
Episode 800 Average Reward (last 100): -200.000
Episode 900 Average Reward (last 100): -200.000
Episode 1000 Average Reward (last 100): -200.000
Episode 1100 Average Reward (last 100): -200.000
Episode 1200 Average Reward (last 100): -200.000
Episode 1300 Average Reward (last 100): -200.000
Episode 1400 Average Reward (last 100): -200.000
Episode 1500 Average Reward (last 100): -200.000
Episode 1600 Average Reward (last 100): -200.000
Episode 1700 Average Reward (last 100): -200.000
Episode 1800 Average Reward (last 100): -200.00

[I 2022-10-03 00:19:20,000] Trial 25 finished with value: -200.0 and parameters: {'gamma': 0.6469940384249309, 'epsilon': 0.134119152413946, 'bins1': 65, 'bins2': 72}. Best is trial 7 with value: -199.26.


...

Episode 20000 Average Reward (last 100): -200.000

TRIAL #32: eps=0.057407355947329625, gamma=0.2802213685906601, bins1=17,bins2=25
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -200.000
Episode 300 Average Reward (last 100): -200.000
Episode 400 Average Reward (last 100): -200.000
Episode 500 Average Reward (last 100): -200.000
Episode 600 Average Reward (last 100): -200.000
Episode 700 Average Reward (last 100): -200.000
Episode 800 Average Reward (last 100): -200.000
Episode 900 Average Reward (last 100): -200.000
Episode 1000 Average Reward (last 100): -200.000
Episode 1100 Average Reward (last 100): -200.000
Episode 1200 Average Reward (last 100): -200.000
Episode 1300 Average Reward (last 100): -200.000
Episode 1400 Average Reward (last 100): -200.000
Episode 1500 Average Reward (last 100): -200.000
Episode 1600 Average Reward (last 100): -200.000
Episode 1700 Average Reward (last 100): -200.000
Episode 1800 Average Reward (last 100): -200

[I 2022-10-03 00:53:10,135] Trial 32 finished with value: -200.0 and parameters: {'gamma': 0.2802213685906601, 'epsilon': 0.057407355947329625, 'bins1': 17, 'bins2': 25}. Best is trial 7 with value: -199.26.


Episode 20000 Average Reward (last 100): -200.000

TRIAL #33: eps=0.0840884898242836, gamma=0.0928297260596422, bins1=36,bins2=54
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -200.000
Episode 300 Average Reward (last 100): -200.000
Episode 400 Average Reward (last 100): -200.000
Episode 500 Average Reward (last 100): -200.000
Episode 600 Average Reward (last 100): -200.000
Episode 700 Average Reward (last 100): -200.000
Episode 800 Average Reward (last 100): -200.000
Episode 900 Average Reward (last 100): -200.000
Episode 1000 Average Reward (last 100): -200.000
Episode 1100 Average Reward (last 100): -200.000
Episode 1200 Average Reward (last 100): -200.000
Episode 1300 Average Reward (last 100): -200.000
Episode 1400 Average Reward (last 100): -200.000
Episode 1500 Average Reward (last 100): -200.000
Episode 1600 Average Reward (last 100): -200.000
Episode 1700 Average Reward (last 100): -200.000
Episode 1800 Average Reward (last 100): -200.0

[W 2022-10-03 00:56:05,733] Trial 33 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-49-dfb51fb42675>", line 14, in train_valuesOn
    (returns, _) = run_montecarloOffP(env_wrapper, 20000, gamma, eps, render=False)
  File "<ipython-input-41-2304de231cae>", line 41, in run_montecarloOffP
    next_state, reward, done, _ = env.step(action)
  File "/usr/local/lib/python3.7/dist-packages/gym/core.py", line 324, in step
    return self.observation(observation), reward, done, info
  File "<ipython-input-25-24e6f88ce49e>", line 44, in observation
    return self.discretizer.to_single_bin(obs)
  File "<ipython-input-25-24e6f88ce49e>", line 13, in to_single_bin
    for i, intervals in enumerate(self.intervals_per_dim)]
  File "<ipython-input-25-24e6f88ce49e>", line 13, in <listcomp>
    for i, int

KeyboardInterrupt: ignored

In [ ]:
study2.optimize(train_valuesOf, n_trials=30)

In [61]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    GAMMA = 0.830525147061507
    EPSILON = 0.05919712699520377

    print("Ambiente Contínuo")
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    print("Monte-Carlo On-Policy")
    env_wrapper = DiscreteObservationWrapper(env, [70,70])
    rewardsOn, QtableOn = run_montecarloOnP(env_wrapper, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))
    filename = f"results/montecarloOnPolicyCont-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'onPolicyC')

    #test_greedy_Q_policy(env, QtableOn, 10, True)


    

Ambiente Contínuo
Monte-Carlo On-Policy
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -200.000
Episode 300 Average Reward (last 100): -200.000
Episode 400 Average Reward (last 100): -200.000
Episode 500 Average Reward (last 100): -200.000
Episode 600 Average Reward (last 100): -200.000
Episode 700 Average Reward (last 100): -200.000
Episode 800 Average Reward (last 100): -200.000
Episode 900 Average Reward (last 100): -200.000
Episode 1000 Average Reward (last 100): -200.000
Episode 1100 Average Reward (last 100): -200.000
Episode 1200 Average Reward (last 100): -200.000
Episode 1300 Average Reward (last 100): -200.000
Episode 1400 Average Reward (last 100): -200.000
Episode 1500 Average Reward (last 100): -200.000
Episode 1600 Average Reward (last 100): -200.000
Episode 1700 Average Reward (last 100): -200.000
Episode 1800 Average Reward (last 100): -200.000
Episode 1900 Average Reward (last 100): -200.000
Episode 2000 Average Reward (last 100)

In [59]:
    env_wrapper = DiscreteObservationWrapper(env, [70,70])
    print("Monte-Carlo Off-Policy")
    env_wrapper = DiscreteObservationWrapper(env, [70,70])
    rewardsOff, QtableOff = run_montecarloOffP(env_wrapper, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffPolicyCont-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'offPolicyC')

    #test_greedy_Q_policy(env, QtableOff, 10, True)
    env.close()

Monte-Carlo Off-Policy
Episode 100 Average Reward (last 100): -200.000
Episode 200 Average Reward (last 100): -200.000
Episode 300 Average Reward (last 100): -200.000
Episode 400 Average Reward (last 100): -200.000
Episode 500 Average Reward (last 100): -200.000
Episode 600 Average Reward (last 100): -200.000
Episode 700 Average Reward (last 100): -200.000
Episode 800 Average Reward (last 100): -200.000
Episode 900 Average Reward (last 100): -200.000
Episode 1000 Average Reward (last 100): -200.000
Episode 1100 Average Reward (last 100): -200.000
Episode 1200 Average Reward (last 100): -200.000
Episode 1300 Average Reward (last 100): -200.000
Episode 1400 Average Reward (last 100): -200.000
Episode 1500 Average Reward (last 100): -200.000
Episode 1600 Average Reward (last 100): -200.000
Episode 1700 Average Reward (last 100): -200.000
Episode 1800 Average Reward (last 100): -200.000
Episode 1900 Average Reward (last 100): -200.000
Episode 2000 Average Reward (last 100): -200.000
Episod

Continuo Ambiente 2 ()

In [101]:
!pip install gym[box2d]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
# ENV_NAME = "CarRacing-v1"  

# env = gym.make(ENV_NAME)
env = gym.make("Pendulum-v1")


In [113]:
def train_valuesOnCar(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    # bins1 = trial.suggest_int('bins1', -1, 1)
    bins2 = trial.suggest_int('bins2', 5.0, 30.0)
    bins3 = trial.suggest_int('bins3', 5.0, 10.0)
   
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma},bins2={bins2},bins3={bins3}")

    # roda o algoritmo e recebe os retornos não-descontados
    env_wrapper = DiscreteObservationWrapper(env, [bins2,bins3])
    (returns, _) = run_montecarloOffP(env_wrapper, 2000, gamma, eps, render=False)
    return sum(returns[-100:])/100

In [114]:

study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_cont_bins_off-policy', 
                            load_if_exists=True)

study.optimize(train_valuesOnCar, n_trials=10) 

[I 2022-10-03 03:20:11,642] Using an existing study with name 'new_MC_offpolice_cont_bins_off-policy' instead of creating a new one.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  """
[W 2022-10-03 03:20:11,796] Trial 10 failed because of the following error: AssertionError()
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  


TRIAL #10: eps=0.07110540318920813, gamma=0.318805085247301,bins2=6,bins3=8


AssertionError: ignored

In [124]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.01
    GAMMA = 0.830525147061507
    EPSILON = 0.05919712699520377
 
    #env_wrapper = DiscreteObservationWrapper(env, [1,80,90])

    env = gym.make("FrozenLake-v1")

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOffP(env, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffPCar-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'offPolicyC-CarRacing')

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

AssertionError: ignored

In [121]:
rewards, Qtable = run_montecarloOnP(env, EPISODES, GAMMA, EPSILON, render=False)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

filename = f"results/montecarloOffPCar-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
plot_result(rewards, r_max_plot,100, 'onPolicyC-CarRacing')

   
env.close()

Episode 100 Average Reward (last 100): 0.000
Episode 200 Average Reward (last 100): 0.000
Episode 300 Average Reward (last 100): 0.000
Episode 400 Average Reward (last 100): 0.010
Episode 500 Average Reward (last 100): 0.040
Episode 600 Average Reward (last 100): 0.040
Episode 700 Average Reward (last 100): 0.070
Episode 800 Average Reward (last 100): 0.090
Episode 900 Average Reward (last 100): 0.090
Episode 1000 Average Reward (last 100): 0.040
Episode 1100 Average Reward (last 100): 0.020
Episode 1200 Average Reward (last 100): 0.060
Episode 1300 Average Reward (last 100): 0.000
Episode 1400 Average Reward (last 100): 0.040
Episode 1500 Average Reward (last 100): 0.000
Episode 1600 Average Reward (last 100): 0.050
Episode 1700 Average Reward (last 100): 0.060
Episode 1800 Average Reward (last 100): 0.050
Episode 1900 Average Reward (last 100): 0.050
Episode 2000 Average Reward (last 100): 0.010
Episode 2100 Average Reward (last 100): 0.040
Episode 2200 Average Reward (last 100): 0.0

# Plot Result

In [150]:
def test_greedy_Q_policy(env, Q, num_episodes=100, render=False, render_wait=0.01, video=None):
    """
    Avalia a política gulosa (greedy) definida implicitamente por uma Q-table.
    Ou seja, executa, em todo estado s, a ação "a = argmax Q(s,a)".
    - env: o ambiente
    - Q: a Q-table (tabela Q) que será usada
    - num_episodes: quantidade de episódios a serem executados
    - render: defina como True se deseja chamar `env.render()` a cada passo
    - render_wait: intervalo de tempo entre as chamadas a `env.render()`
    - video 
    
    Retorna:
    - um par contendo o valor escalar do retorno médio por episódio e 
       a lista de retornos de todos os episódios
    """
    episode_returns = []
    total_steps = 0
    for i in range(num_episodes):
        print(f"Episode {i+1}")
        obs = env.reset()
        if render:
            env.render()
            time.sleep(render_wait)
        if video is not None:
            video.capture_frame()
        done = False
        episode_returns.append(0.0)
        while not done:
            action = np.argmax(Q[obs])
            obs, reward, done, _ = env.step(action)
            if render:
                env.render()
                time.sleep(render_wait)
            if video is not None:
                video.capture_frame()
            total_steps += 1
            episode_returns[-1] += reward
        print("- retorno:", episode_returns[-1])
    mean_return = round(np.mean(episode_returns), 1)
    print("Retorno médio (por episódio):", mean_return, end="")
    print(", episódios:", len(episode_returns), end="")
    print(", total de passos:", total_steps)
    show_state(env,total_steps)
    if video is not None:
        video.close()
    return mean_return, episode_returns

In [40]:
def smooth(data, window):
  data = np.array(data)
  n = len(data)
  y = np.zeros(n)
  for i in range(n):
    start = max(0, i-window+1)
    y[i] = data[start:(i+1)].mean()
  return y

def plot_result(returns, ymax_suggested=None, window=100, filename=None):
    '''Exibe um gráfico "retornos x recompensas", fazendo a média a cada 100 retornos, para suavizar.     
    Se o parâmetro filename for fornecido, salva o gráfico em arquivo ao invés de exibir.
    
    Parâmetros:
    - returns: lista de retornos a cada episódio
    - ymax_suggested (opcional): valor máximo de retorno (eixo y), se tiver um valor máximo conhecido previamente
    - filename: indique um nome de arquivo, se quiser salvar a imagem do gráfico; senão, o gráfico será apenas exibido
    '''
    plt.figure(figsize=(14,8))
    smoothed_returns = smooth(returns, window)
    xvalues = np.arange(1, len(returns)+1)
    plt.plot(xvalues, smoothed_returns)
    plt.xlabel('Episódios')
    plt.ylabel('Retorno')
    if ymax_suggested is not None:
        ymax = np.max([ymax_suggested, np.max(smoothed_returns)])
        plt.ylim(top=ymax)
    plt.title(f"Retorno médio a cada {window} episódios")
    if filename is None:
        plt.show()
    else:
        plt.savefig(filename)
        print("Arquivo salvo:", filename)
    plt.close()


# Exibe o ambiente

In [62]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder

In [152]:
ENV_NAME = 'MountainCar-v0'

In [153]:
env = gym.make(ENV_NAME)

In [148]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [149]:
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [154]:
#record_video(ENV_NAME, model, video_length=1000, prefix='monte-car-off-police')

videoOf = VideoRecorder(env, "monte-car-off-police.mp4")
#videoOn = VideoRecorder(env, "monte-car-on-police.mp4")
#test_greedy_Q_policy(env, QtableOn, 10, True,videoOn)
#render_mp4('video_offpolicy')

test_greedy_Q_policy(env, QtableOn, 10, True,videoOf)
render_mp4('monte-car-off-police.mp4')
#show_videos('videos', prefix='monte-car-off-police')

ValueError: ignored